In [1]:
import numpy as np
f1 = open(".csv","rb")
features_selected =np.loadtxt(f1,delimiter=',',skiprows=0)
f1.close()
features_selected = np.array(features_selected)
#features = np.column_stack((k_mer,theta_list,DSA,ADPM)) 
print(features_selected.shape)

(644, 220)


In [2]:
# Read labels
target= np.load("target.npy")
print(target.shape)

(644,)


In [3]:
#处理不平衡数据集
from imblearn.over_sampling import SMOTE
X_resampled_smote, y_resampled_smote = SMOTE(random_state = 42).fit_sample(features_selected, target)
features_selected = X_resampled_smote
target = y_resampled_smote
print(features_selected.shape)
print(target.shape)

(1668, 220)
(1668,)


In [4]:
from sklearn.preprocessing import MinMaxScaler


In [5]:
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline

from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier
#pipe = Pipeline([("scaler", MinMaxScaler()), 
#                 ("svm",SVC(C=2, gamma=0.5, decision_function_shape="ovo",kernel="rbf",random_state = 21,probability=True))])


gnb=GaussianNB()
dt = DecisionTreeClassifier(random_state=21)
knn=KNeighborsClassifier(n_neighbors=1,leaf_size=1)
rf=RandomForestClassifier(n_estimators=145,random_state=28,max_features='auto')
bagging = BaggingClassifier(n_estimators=195,random_state = 21)
adaboost = AdaBoostClassifier (n_estimators=195)

pipe = Pipeline([("scaler", MinMaxScaler()), 
                 ("adaboost",AdaBoostClassifier (n_estimators=195))])
loo = LeaveOneOut()

In [6]:
import numpy as np
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score

# Calculate the confusion matrix
confusion_matrix_myself = np.zeros((4,4))

def coufusion_matrix_function(y_true, y_pred):
    for num in range(len(y_true)):
        i = y_true[num]
        j = y_pred[num]
        confusion_matrix_myself[i][j] += 1
    return 1
confusion_matrix_score = make_scorer(coufusion_matrix_function, greater_is_better=True)

confusion_matrix_scores = cross_val_score(pipe, features_selected, target, 
                                  cv = loo, scoring = confusion_matrix_score)  

In [7]:
print(confusion_matrix_myself)

[[278. 121.  17.   1.]
 [151. 237.  16.  13.]
 [ 20.  26. 371.   0.]
 [ 10.  64.   4. 339.]]


In [8]:
# Calculate the metric: sensitivity
def sensitivity_myself(confusion_matrix, i):
    N_plus_minus_i = sum(confusion_matrix[i]) - confusion_matrix[i,i]
    N_plus_i = sum(confusion_matrix[i])
    Sn_i = 1 - float(N_plus_minus_i)/N_plus_i
    return Sn_i

In [9]:
# Calculate the metric: specificity
def specificity_myself(confusion_matrix, i):
    matrix_sum = sum(sum(j) for j in confusion_matrix)
    N_minus_plus_i = sum(confusion_matrix[:,i])-confusion_matrix[i,i]
    N_minus_i = matrix_sum - sum(confusion_matrix[i])
    Sp_i = 1 - float(N_minus_plus_i)/N_minus_i
    return Sp_i

In [10]:
# Calculate the Sn(i)
for i in range(4):
    print(sensitivity_myself(confusion_matrix_myself, i))

0.6666666666666667
0.5683453237410072
0.8896882494004796
0.8129496402877698


In [11]:
# Calculate the Sp(i)
for i in range(4):
    print(specificity_myself(confusion_matrix_myself, i))

0.8553157474020783
0.8313349320543565
0.9704236610711431
0.9888089528377298


In [12]:
# Calculate the metric: Matthew's correlation coefficient
import math
def mcc_myself(confusion_matrix, i):
    matrix_sum = sum(sum(j) for j in confusion_matrix)
    N_plus_minus_i = sum(confusion_matrix[i]) - confusion_matrix[i,i]
    N_plus_i = sum(confusion_matrix[i])
    N_minus_plus_i = sum(confusion_matrix[:,i])-confusion_matrix[i,i]
    N_minus_i = matrix_sum - sum(confusion_matrix[i])
    part_1 =float(N_plus_minus_i)/N_plus_i + float(N_minus_plus_i)/N_minus_i
    part_2 =1+float(N_minus_plus_i-N_plus_minus_i)/N_plus_i
    part_3 =1+float(N_plus_minus_i-N_minus_plus_i)/N_minus_i
    mcc_i = float(1 - part_1)/math.sqrt(part_2*part_3)
    return mcc_i

In [13]:
# Calculate the MCC(i)
for i in range(4):
    print(mcc_myself(confusion_matrix_myself, i))

0.5060961816755475
0.3904725014581743
0.8664356032806034
0.8499435383754407


In [14]:
# Calculate the metric: overall accuracy
def accuracy_myself(confusion_matrix):
    num_i = 0
    for i in range(4):
        N_plus_i = sum(confusion_matrix[i])
        N_plus_minus_i = sum(confusion_matrix[i]) - confusion_matrix[i,i]
        num_i += N_plus_i - N_plus_minus_i
    return float(num_i)/1668

In [15]:
# Calculate the OA
print(accuracy_myself(confusion_matrix_myself))

0.7344124700239808
